In [20]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=10

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],0:2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=3
p=floor(Int64,n/u)

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
f=sum(generate_random_poly(I[j]) for j in 1:p)


g=[-(x.^2).+1/u;(x.^2).-0.5/u]



m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)

#=randx=[2*rand(length(I[j])).-1 for j in 1:p]# create a feasible solution
randx=[randx[j]/norm(randx[j]) for j in 1:p]=#

randx=2*rand(n).-1
randx=randx./sqrt(u)

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)
println("====================")

k=2

println("Relaxed order: k=",k)
println("====================")

***Problem setting***
Number of variable: n=10
Number of inequality constraints: m=20
Number of equality constraints: l=2
Relaxed order: k=2


In [21]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

In [22]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_CS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k;EigAlg="Arpack",maxit=1e10,tol=1e-3,UseEq=false)

------------------------------------------------------
The clique sizes of varibles:
[5, 4, 3]
[1, 1, 1]
------------------------------------------------------
[5, 4, 3]
  Number of cliques: p=3
  Largest clique size: u=5


  Largest constant trace: a=6.111111111111109
  Number of blocks: omega=25
  Number of equality consraints: zeta=582
  Size of the largest block: s^max=21
Modeling time:
  3.538372 seconds (3.15 M allocations: 159.191 MiB, 1.92% gc time)
iter=1.0   val=-22.951431470859756   gap=0.04003495619378093   feas=12.357189178259887
iter=2.0   val=-11.305426185343789   gap=0.6103639032854061   feas=6.469395368704147
iter=4.0   val=-9.601760391353338   gap=0.41653116502981147   feas=4.3615135931705336
iter=8.0   val=-6.570558323414767   gap=0.2516477757191972   feas=2.6135603333230604
iter=16.0   val=-6.247366532484119   gap=0.3296523577780061   feas=1.4553057511829328
iter=32.0   val=-5.379942535512352   gap=0.2616743589163465   feas=0.7625797889430839
iter=64.0   val=-5.307860972705002   gap=0.2267565695414997   feas=0.3997784478138315
iter=128.0   val=-5.231641816542352   gap=0.07783498281246702   feas=0.21055659607528268
iter=256.0   val=-5.261678253399462   gap=0.029341385082017616   feas=0.

-4.771461320285669

In [23]:
using TSSOS


@time opt,sol,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS="block",CTP=false);

------------------------------------------------------
The clique sizes of varibles:
[5, 4, 3]
[1, 1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 221             
  Cones                  : 0               
  Scalar variables       : 43              
  Matrix variables       : 26              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. 